<a href="https://colab.research.google.com/github/hydrosolutions/pycaret_3_forecasting/blob/main/x_reg_forecasting_issue_pycaret_302.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyCaret 3.0.2 Time Series Experiment with External Regressors - 
Issue - Error / Unexpected Behavior

## Libraries

In [1]:
def what_is_installed():
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except ModuleNotFoundError:
    !pip install pycaret[full]
    what_is_installed()


System:
    python: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
executable: /usr/bin/python3
   machine: Linux-5.15.107+-x86_64-with-glibc2.31

PyCaret required dependencies:
                 pip: 23.1.2
          setuptools: 67.7.2
             pycaret: 3.0.2
             IPython: 7.34.0
          ipywidgets: 7.7.1
                tqdm: 4.65.0
               numpy: 1.22.4
              pandas: 1.5.3
              jinja2: 3.1.2
               scipy: 1.10.1
              joblib: 1.2.0
             sklearn: 1.2.2
                pyod: 1.0.9
            imblearn: 0.10.1
   category_encoders: 2.6.1
            lightgbm: 3.3.5
               numba: 0.56.4
            requests: 2.27.1
          matplotlib: 3.7.1
          scikitplot: 0.3.7
         yellowbrick: 1.5
              plotly: 5.13.1
             kaleido: 0.2.1
         statsmodels: 0.13.5
              sktime: 0.17.0
               tbats: 1.1.3
            pmdarima: 2.0.3
              psutil: 5.9.5

PyCaret optional depend

In [4]:
import numpy as np
import pandas as pd
from pycaret.time_series import TSForecastingExperiment

## Data & Parameters

### Data

We load two types of different future external regressor data. For the 'fut_data_xreg_exp' data, we compute long lagged rolling means from the prep_data_exp. For comparison, we infer these data using predictions in the case of the 'fut_data_xreg_inferred' dataframe. This is just to demonstrate the unexpected behavior below. 

In [30]:
# Loading relevand data
## data for training and testing
prep_data_exp = pd.read_csv('https://raw.githubusercontent.com/hydrosolutions/pycaret_3_forecasting/main/data/prep_data_exp.csv?token=GHSAT0AAAAAACB7VEENNMOTFDFG6SWFNV5YZEG2RNQ')
## future external regressors
fut_data_xreg_exp = pd.read_csv('https://raw.githubusercontent.com/hydrosolutions/pycaret_3_forecasting/main/data/x_reg_data_exp.csv?token=GHSAT0AAAAAACB7VEENTKVKNA75X6SJTW3CZEG2SRA')
display(fut_data_xreg_exp)
## future external regressors (predicted)
fut_data_xreg_inferred_exp = pd.read_csv('https://raw.githubusercontent.com/hydrosolutions/pycaret_3_forecasting/main/data/x_reg_inferred_data_exp.csv?token=GHSAT0AAAAAACB7VEEMXRRXFEWOYO6LVMSSZEG33HQ')
display(fut_data_xreg_inferred_exp)

,month,year,q_lag_roll_6,q_lag_roll_12
0,1,2023,617.833333,388.750000
1,2,2023,676.833333,389.166667
2,3,2023,700.666667,390.333333
3,4,2023,648.833333,412.416667
4,5,2023,533.000000,419.833333
5,6,2023,403.166667,440.583333


,month,year,q_lag_roll_6,q_lag_roll_12
0,1.0,2023.0100,641.9767,387.5878
1,2.0,2023.0022,722.9847,386.3706
2,3.0,2023.0019,757.1390,385.4575
3,4.0,2023.0016,742.1414,384.2411
4,5.0,2023.0013,648.8025,383.3272
5,6.0,2023.0009,493.0944,382.1117


### Parameters

In [9]:
exp_fh = 6

## Experiment
### Setup

In [11]:
exp = TSForecastingExperiment()
exp.setup(data = prep_data_exp,
                    target = 'q',
                    fh = exp_fh,
                    numeric_imputation_target = "ffill",
                    numeric_imputation_exogenous = "ffill", 
                    session_id = 123,
                    log_experiment = False,
                    profile = False,
                    experiment_name = 'ts_exp_llag')
exp

,Description,Value
0,session_id,123
1,Target,q
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(264, 5)"
5,Transformed data shape,"(264, 5)"
6,Transformed train set shape,"(258, 5)"
7,Transformed test set shape,"(6, 5)"
8,Rows with missing values,8.7%
9,Fold Generator,ExpandingWindowSplitter


### Compare Models

In [12]:
best_model = exp.compare_models(turbo=False)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,0.6854,0.5701,64.6469,93.2833,0.1339,0.1394,0.8746,0.5900
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,0.6890,0.5674,64.9673,92.8250,0.1307,0.1370,0.8771,0.6667
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,0.7264,0.6222,68.5170,101.8493,0.1357,0.1348,0.8527,4.8300
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.7281,0.5800,68.6524,94.8765,0.1447,0.1554,0.8709,0.6067
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.7281,0.5800,68.6523,94.8764,0.1447,0.1554,0.8709,0.4300
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,0.7287,0.5801,68.7162,94.8932,0.1450,0.1557,0.8708,0.4067
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.7325,0.5810,69.0671,95.0384,0.1464,0.1574,0.8705,0.6300
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.7325,0.5810,69.0659,95.0379,0.1464,0.1574,0.8705,0.4133
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.7458,0.6419,70.3210,105.0544,0.1371,0.1367,0.8450,0.9700
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,0.7462,0.6063,70.3579,99.1986,0.1473,0.1540,0.8620,0.6133


Processing:   0%|          | 0/105 [00:00<?, ?it/s]

### Tuning of Best Model

In [13]:
best_tuned_model = exp.tune_model(best_models)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,239.0000,0.9855,0.7875,93.4623,129.8500,0.1814,0.1813,0.7756
1,245.0000,0.1723,0.1525,16.2396,24.9439,0.0614,0.0564,0.9832
2,251.0000,0.8896,0.7701,83.3992,125.0081,0.1561,0.1772,0.8650
Mean,nan,0.6825,0.5700,64.3671,93.2673,0.1330,0.1383,0.8746
SD,nan,0.3628,0.2953,34.2783,48.3524,0.0516,0.0579,0.0850


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   19.2s finished


### Plotting 

In [14]:
exp.plot_model(best_tuned_model)

This looks very well so far. Now we finalize and then try forecacsting with external regressors.

### Model Finalization

In [16]:
final_model = exp.finalize_model(best_tuned_model)

In [17]:
final_model

ForecastingPipeline(steps=[('transformer_exogenous',
                            TransformerPipeline(steps=[('numerical_imputer',
                                                        Imputer(method='ffill',
                                                                random_state=123))])),
                           ('forecaster',
                            TransformedTargetForecaster(steps=[('transformer_target',
                                                                TransformerPipeline(steps=[('numerical_imputer',
                                                                                            Imputer(method='ffill',
                                                                                                    random_state=123))])),
                                                               ('model',
                                                                BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [12,
                                                                                                                                        11,
                                                                                                                                        10,
                                                                                                                                        9,
                                                                                                                                        8,
                                                                                                                                        7,
                                                                                                                                        6,
                                                                                                                                        5,
                                                                                                                                        4,
                                                                                                                                        3,
                                                                                                                                        2,
                                                                                                                                        1]},
                                                                                                                   n_jobs=1)],
                                                                                    regressor=BayesianRidge(alpha_1=2.5583491834921515e-07,
                                                                                                            alpha_2=0.0007811241491018094,
                                                                                                            fit_intercept=False,
                                                                                                            lambda_1=4.764729145527039e-08,
                                                                                                            lambda_2=3.6938492594950892e-06),
                                                                                    sp=12,
                                                                                    window_length=15))]))])

## Forecasting
### Prediction Wrapper

In [18]:
def safe_predict(exp, model):
    """Prediction wrapper for demo purposes."""
    try: 
        exp.predict_model(model)
    except ValueError as exception:
        print(exception)
        exo_vars = exp.exogenous_variables
        print(f"{len(exo_vars)} exogenous variables (X) needed in order to make future predictions:\n{exo_vars}")

### Test

In [19]:
safe_predict(exp,final_model)

Model was trained with exogenous variables but you have not passed any for predictions. Please pass exogenous variables to make predictions.
4 exogenous variables (X) needed in order to make future predictions:
['month', 'year', 'q_lag_roll_6', 'q_lag_roll_12']


### Passing External Regressors

In [26]:
q_forecast = exp.predict_model(final_model, fh = exp_fh, X = fut_data_xreg_exp)
q_forecast.head()

,y_pred
264,196.6660
265,188.4316
266,208.7768
267,402.8932
268,742.6616


## Unexpected Behavior

We have two issues here. The first one in relation to the forecasting horizon, the second one in relation to the actual forecasts.

### A. Length of Forecasting Horizon

Forecast is produced even if fh > number of x_reg observations. How can that be?

In [27]:
q_forecast_fh_60 = exp.predict_model(final_model, fh = 60, X = fut_data_xreg_exp)
q_forecast_fh_60.head(12)

,y_pred
264,196.6660
265,188.4316
266,208.7768
267,402.8932
268,742.6616
269,1027.4327
270,837.5109
271,591.1673
272,354.5035
273,264.8502


Now, for demo purposes, I pass only 1 data point as X key

In [31]:
q_forecast_fh_60 = exp.predict_model(final_model, fh = 60, X = fut_data_xreg_exp.head(1))
q_forecast_fh_60.head(12)

,y_pred
264,196.6660
265,188.4316
266,208.7768
267,402.8932
268,742.6616
269,1027.4327
270,837.5109
271,591.1673
272,354.5035
273,264.8502


This brings me to the second unexpected behavior.

### B. Prediction Results



Prediction results are always the same, independent of which dataset is passed as X key.

In [28]:
q_forecast_xreg_inferred = exp.predict_model(final_model, fh = exp_fh, X = fut_data_xreg_inferred_exp)
q_forecast_xreg_inferred.head()

,y_pred
264,196.6660
265,188.4316
266,208.7768
267,402.8932
268,742.6616
